### Update information: Input required
#### Note: quotation marks should be included. Change values inside of the quotes.

In [1]:
AY = '2022-2023'
CO = 'c/o 2025'
Block = 'Brain and Behavior'

That's it. Now you can sit back and run the code! 

Note: There are exceptional questions that need modification. Please refer to the SOP.

### Block Data Automation

In [2]:
# initiate
import numpy as np
import pandas as pd
import re
from datetime import datetime
import datetime
import time
import warnings
warnings.simplefilter("ignore")

# start timer
tic = time.time()

# import dataset
read = pd.read_excel('input.xlsx', engine='openpyxl')

#filter in applicable data only: "0/IP Address" rows only
readfile = read[(read.Status.isin(['Response Type', 0]))]

# subset all the survey questions
q_subset = readfile.loc[:,readfile.columns.str.contains('Q.*')]

# create a list of columns
q_cols_subset = list(q_subset.columns)

# create a dataframe for stats
df1 = pd.DataFrame([])

for j in q_cols_subset:

    c = q_subset[j][1:,].value_counts().rename_axis('unique_values').reset_index(name='counts')

    cc = {}
    
    for i in range(1,6):
        if i in list(c.unique_values):
            cn = c[c.unique_values == i]['counts']

            cc.update(                {
                    'C'+str(i):int(cn)
                }            )
            
        #filter out invalid values (not a number between 1-5: Text or other coded value for N/A)
        else:
            cc.update(                {
                    'C'+str(i):0
                }              )


    dd = pd.DataFrame([cc])
    dd['N'] = dd.C1 + dd.C2 + dd.C3 + dd.C4 + dd.C5
    dd['Sumproduct'] = dd.C1 * 1 + dd.C2 * 2 + dd.C3 * 3 + dd.C4 * 4 + dd.C5 * 5
    dd['Mean'] = (dd.C1 * 1 + dd.C2 * 2 + dd.C3 * 3 + dd.C4 * 4 + dd.C5 * 5)/dd.N
    df1 = pd.concat([df1, dd], axis=0, ignore_index=True)
    df1 = df1[['Mean', 'N','Sumproduct']]
#df1

# create a dataframe for question by extracting text
sequence2=0
q0 = []
for l in q_cols_subset:
    q_text_q = str(q_subset[l][0])  
    q0.append(q_text_q)


df_q1 = pd.DataFrame(q0)
df_q1.columns = ['Question']
#df_q1

# concat dataframes
df4 = pd.concat([df_q1, df1[['Mean','N']]], axis=1)
#df4

# delete unapplicable rows/qualitative questions
df5 = df4[df4['N'] > 0]

# populate columns for AY, Class, Block
df5['Year'] = AY
df5['Class'] = CO
df5['Block'] = Block
df5['Response'] = 'Mean'
df5['Value'] = ''
df5['Section'] = ''
df5 = df5[['Year', 'Block', 'Section', 'Question', 'Response', 'Value', 'Mean', 'N', 'Class']]
#df5

# export
df5.to_excel('output_' + str(datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")) + '.xlsx', index = False)

# end timer             
toc = time.time() - tic
print('All tasks completed in %.2f s.' % (toc))

All tasks completed in 1.06 s.
